# Excel Export UDF

Xcalar allows your data to be exported to a variety of external target types ranging from files on your local system to databases, Kafka, Hadoop and REST APIs. By default, Xcalar will export a table to the Default <b>Export Target</b> which is in the shared file system that is mounted on all Xcalar cluster nodes. In this tutorial we will look into creating custom export targets leveraging UDFs.

Note: To integrate an Export UDF you will need to create an Export Target which requires an admin account privilege.

<HTML>
<br>
<div style="background-color : blue; color : white
    width: 284px;
    padding: 20px 20px 20px 100px;
    border: 1px solid #BFBFBF;
    background-color: white;box-shadow: 0px 0px 0px 0px #aaaaaa; position: relative;"><font style="font-size:20px">
More Information on Exporting</font>
    <br>For information on exporting a table in Xcalar refer to [Exporting a table](https://www.xcalar.com/documentation/help/XD/1.3.1/Content/C_AdvancedTasks/G_ExportTable.htm).
    <br>
    For more information on writing an export UDF and creating an Export Target see [Example of using a UDF when exporting](https://www.xcalar.com/documentation/help/XD/1.3.1/Content/D_Reference/H1_UdfExamples.htm#Example).
    <br>This video tutorial [How-to Series (UDFs) - Exporting to Kafka](https://www.youtube.com/watch?v=c7FKfsct8PU) on the Xcalar YouTube channel might also be useful.
    <img src="xi-questionmark_yellow.png" 
         style="position: absolute;top: 20px;left: 30px;width:40px ;height:40px" />
</div>
</HTML>


### Export UDFs
By default, tables are exported in CSV format, however, you can export in any format and to any target using Export UDFs. Before implementing our example UDF lets cover some basics of export UDFs.

An Export UDF is required to have a function named ‘main’ that takes one argument, in our example we call it ‘inStruct’. The ‘inStruct’ argument is a json string that has two attributes, ‘filePath’  and ‘fileContent’. ‘filePath’ contains the path to the target location and ‘fileContent’ is the data being exported. More specifically:

* <i>inStruct[“filePath”]</i> contains the export file system path consisting of the path specified in the the export target (see **Creating the export target** section [here](https://www.xcalar.com/documentation/help/XD/1.3.1/Content/D_Reference/H1_UdfExamples.htm?Highlight=%22export%20target%22#Example")) and the filename as specified during the export operation.
* <i>inStruct[“fileContent”]</i> contains the exported content in CSV format. 

The code below will implement a Export UDF that converts and saves your Xcalar table in Excel format.

In [1]:
import openpyxl
import json
import os

def main(inStruct):
    # Get the fileName and fileContents from the argument
    s = json.loads(inStruct)
    fileName = s["filePath"]
    fileContents = s["fileContents"]

    # Create excel workbook
    wb = openpyxl.Workbook()
    # File name removes the .csv and replaces it with .xlsx
    folderName = fileName[:fileName.rfind("/")];

    # Create the folder if it doesn't already exist
    if not os.path.exists(folderName):
        os.makedirs(folderName)

    fileName = fileName.replace(".csv", ".xlsx")

    ws = wb.active

    fieldDelim = "\t" # Change this to match the delimiter that you chose in
                      # the export panel
    rowNum = 0
    for row in fileContents.split("\n"):
        rowNum += 1
        colNum = 0
        for col in row.split(fieldDelim):
            colNum += 1
            ws.cell(column=colNum, row=rowNum, value=str(col))

    wb.save(filename=fileName)

### Setting your UDF as an Export Target

To deploy this UDF you will need to specify a new Export Target as described in the **Creating an export target** section [here](https://www.xcalar.com/documentation/help/XD/1.3.1/Content/C_AdvancedTasks/G_ExportTable.htm?Highlight="export%20target"). Remember that this task requires admin privileges.


<html>
 Back to <a href="./0%20-%20Introduction.ipynb" target="_self">Introduction</a><br>
</html>